#  CV3DST  ReID
- to train a small ReID dataset with cross-entropy and triplet-loss.

#### Install and import Python libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
reid_root_dir = ".."
root_dir = '..'
sys.path.append(os.path.join(reid_root_dir, 'src'))


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

from torch.utils.data import DataLoader

from tracker.data_track import MOT16Sequences
from tracker.data_obj_detect import MOT16ObjDetect
from tracker.object_detector import FRCNN_FPN
from tracker.tracker import Tracker, ReIDTracker
from tracker.utils import (plot_sequence, evaluate_mot_accums, get_mot_accum,
                           evaluate_obj_detect, obj_detect_transforms)
# Load helper code
from market.datamanager import ImageDataManager
from market.models import build_model
from market import utils
from market import metrics

import torch
from torch.nn import functional as F
from scipy.optimize import linear_sum_assignment as linear_assignment

import motmetrics as mm
mm.lap.default_solver = 'lap'

/tmp/ipykernel_26956/752511971.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [6]:
seed = 12345
seq_name = 'MOT16-reid'  # We recommend to use this subset.
data_dir = os.path.join(root_dir, 'data/MOT16')
output_dir = os.path.join(root_dir, 'output')

## Setup

In [7]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

# Training a ReID Network

train a simple ReID network on the Market data. we will use a ResNet34/ResNet50 neural network that extracts features from an input image. 

Next, create the the DataManager for the Market dataset that will provide the train and test sets:

In [8]:
datamanager = ImageDataManager(root=reid_root_dir, height=256,width=128, batch_size_train=32, 
                               workers=2, transforms=['random_flip', 'random_crop'])
train_loader = datamanager.train_loader
test_loader = datamanager.test_loader

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

Now, let's create a resnet34 model and move it to the GPU.

In [13]:
#model = build_model('resnet34', datamanager.num_train_pids, loss='softmax', pretrained=True)
#model = model.cuda()

trainable_params = model.parameters()

In [21]:
from torch import nn


In [31]:
import antialiased_cnns
model = antialiased_cnns.resnet34(pretrained=True) 
model.fc = nn.Linear(512, 751)
trainable_params = model.parameters()
model = model.cuda()

For training the network, we now need to choose an optimizer and learning rate scheduler.

In [32]:
optimizer = torch.optim.Adam(trainable_params, lr=0.0003, 
                             weight_decay=5e-4, amsgrad=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)

The network will be trained on a cross-entropy loss, i.e., the network needs to classify each image to it's identity class. For $n$ different people, we will have $n$ different classes.

During evaluation, we ignore the last classification layer and work on the extracted $feat$-dimensional features. This feature vector should be very similar for the same instance, and not similar for different instances.

In the following, you have to implement two distance measurements:
- Euclidian squared distance.
- Cosine similarity.

You are not allowed to change the interface of the function. Please have a look at the [Pytorch documentation](https://pytorch.org/docs/stable/index.html).

In [33]:

def euclidean_squared_distance(input1, input2):
    """Computes euclidean squared distance.
    Args:
        input1 (torch.Tensor): 2-D feature matrix.
        input2 (torch.Tensor): 2-D feature matrix.
    Returns:
        torch.Tensor: distance matrix.
    """
    distmat = torch.cdist(input1, input2, p=2.0) 
    return distmat**2



In [34]:
def cosine_distance(input1, input2):
    """Computes cosine distance.
    Args:
        input1 (torch.Tensor): 2-D feature matrix (m x feat).
        input2 (torch.Tensor): 2-D feature matrix (n x feat).
    Returns:
        torch.Tensor: distance matrix (m x n).
    """

    # Given that cos_sim(u, v) = dot(u, v) / (norm(u) * norm(v))
    #                          = dot(u / norm(u), v / norm(v))
    # We fist normalize the rows, before computing their dot products via transposition:
    norm1 = input1.norm(dim=1)[:, None]
    norm2 = input2.norm(dim=1)[:, None]
    input1_norm = input1/norm1
    input2_norm = input2/norm2
    cosine_similarity = torch.mm(input1_norm, input2_norm.t())
    distmat = 1 - cosine_similarity
    return distmat

With the implemented distance measure, we can now implement the evaluation function. We extract features for the query set and for the gallery set and then build a distance matrix based on your implemented distance measure.

In [35]:
metric_fn = cosine_distance  # cosine_distance or euclidean_squared_distance
def evaluate(model, test_loader, ranks=[1, 5, 10, 20]):
    with torch.no_grad():
        model.eval()
        print('Extracting features from query set...')
        q_feat, q_pids, q_camids = utils.extract_features(model, test_loader['query'])
        print('Done, obtained {}-by-{} matrix'.format(q_feat.size(0), q_feat.size(1)))

        print('Extracting features from gallery set ...')
        g_feat, g_pids, g_camids = utils.extract_features(model, test_loader['gallery'])
        print('Done, obtained {}-by-{} matrix'.format(g_feat.size(0), g_feat.size(1)))
        
        distmat = metrics.compute_distance_matrix(q_feat, g_feat, metric_fn=metric_fn)
        distmat = distmat.numpy()

        print('Computing CMC and mAP ...')
        cmc, mAP = metrics.eval_market1501(
            distmat,
            q_pids,
            g_pids,
            q_camids,
            g_camids,
            max_rank=50
        )

        print('** Results **')
        print('mAP: {:.1%}'.format(mAP))
        print('CMC curve')
        for r in ranks:
            print('Rank-{:<3}: {:.1%}'.format(r, cmc[r - 1]))
        return cmc[0], mAP

Finally, we can implement the training logic.

In [36]:
MAX_EPOCH = 30
EPOCH_EVAL_FREQ = 5
PRINT_FREQ = 50

num_batches = len(train_loader)
criterion = torch.nn.CrossEntropyLoss() 

for epoch in range(MAX_EPOCH):
    losses = utils.MetricMeter()
    batch_time = utils.AverageMeter()
    end = time.time()
    model.train()
    for batch_idx, data in enumerate(train_loader):
        # Predict output.
        imgs, pids = data['img'].cuda(), data['pid'].cuda()
        output = model(imgs)
        # Compute loss.
        loss = criterion(output, pids)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time.update(time.time() - end)
        losses.update({'Loss': loss})
        if (batch_idx + 1) % PRINT_FREQ == 0:
            utils.print_statistics(batch_idx, num_batches, epoch, MAX_EPOCH, batch_time, losses)
        end = time.time()
        
    if (epoch + 1) % EPOCH_EVAL_FREQ == 0 or epoch == MAX_EPOCH - 1:
        rank1, mAP = evaluate(model, test_loader)
        print('Epoch {0}/{1}: Rank1: {rank}, mAP: {map}'.format(
                    epoch + 1, MAX_EPOCH, rank=rank1, map=mAP))

epoch: [1/30][50/404]	time 0.071 (0.124)	eta 0:24:56	Loss 6.8585 (6.7444)	
epoch: [1/30][100/404]	time 0.072 (0.098)	eta 0:19:38	Loss 6.0864 (6.4957)	
epoch: [1/30][150/404]	time 0.072 (0.090)	eta 0:17:51	Loss 5.5497 (6.1983)	
epoch: [1/30][200/404]	time 0.072 (0.085)	eta 0:16:56	Loss 4.9574 (5.8745)	
epoch: [1/30][250/404]	time 0.074 (0.083)	eta 0:16:23	Loss 3.7464 (5.5444)	
epoch: [1/30][300/404]	time 0.073 (0.081)	eta 0:15:59	Loss 3.5259 (5.2428)	
epoch: [1/30][350/404]	time 0.074 (0.080)	eta 0:15:41	Loss 2.5419 (4.9530)	
epoch: [1/30][400/404]	time 0.074 (0.079)	eta 0:15:28	Loss 3.2357 (4.6809)	
epoch: [2/30][50/404]	time 0.074 (0.077)	eta 0:14:54	Loss 2.0897 (2.1703)	
epoch: [2/30][100/404]	time 0.074 (0.075)	eta 0:14:32	Loss 1.4728 (2.0618)	
epoch: [2/30][150/404]	time 0.073 (0.075)	eta 0:14:23	Loss 2.0705 (1.9614)	
epoch: [2/30][200/404]	time 0.074 (0.074)	eta 0:14:16	Loss 1.7814 (1.8691)	
epoch: [2/30][250/404]	time 0.074 (0.074)	eta 0:14:11	Loss 1.4400 (1.7959)	
epoch: [2/30][

epoch: [13/30][200/404]	time 0.074 (0.075)	eta 0:08:53	Loss 0.1946 (0.1146)	
epoch: [13/30][250/404]	time 0.074 (0.075)	eta 0:08:48	Loss 0.1278 (0.1210)	
epoch: [13/30][300/404]	time 0.074 (0.075)	eta 0:08:43	Loss 0.2987 (0.1223)	
epoch: [13/30][350/404]	time 0.075 (0.075)	eta 0:08:39	Loss 0.1079 (0.1284)	
epoch: [13/30][400/404]	time 0.075 (0.075)	eta 0:08:35	Loss 0.0890 (0.1332)	
epoch: [14/30][50/404]	time 0.076 (0.079)	eta 0:08:56	Loss 0.1083 (0.1320)	
epoch: [14/30][100/404]	time 0.078 (0.077)	eta 0:08:40	Loss 0.1272 (0.1274)	
epoch: [14/30][150/404]	time 0.075 (0.076)	eta 0:08:32	Loss 0.1213 (0.1189)	
epoch: [14/30][200/404]	time 0.075 (0.076)	eta 0:08:26	Loss 0.1392 (0.1151)	
epoch: [14/30][250/404]	time 0.075 (0.076)	eta 0:08:21	Loss 0.0552 (0.1154)	
epoch: [14/30][300/404]	time 0.075 (0.076)	eta 0:08:16	Loss 0.1965 (0.1179)	
epoch: [14/30][350/404]	time 0.074 (0.075)	eta 0:08:11	Loss 0.2056 (0.1237)	
epoch: [14/30][400/404]	time 0.075 (0.075)	eta 0:08:07	Loss 0.1136 (0.1258)	


epoch: [25/30][300/404]	time 0.075 (0.075)	eta 0:02:40	Loss 0.0594 (0.0917)	
epoch: [25/30][350/404]	time 0.075 (0.075)	eta 0:02:36	Loss 0.0388 (0.0950)	
epoch: [25/30][400/404]	time 0.075 (0.075)	eta 0:02:32	Loss 0.1237 (0.0988)	
Extracting features from query set...
Done, obtained 3368-by-751 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-751 matrix
Computing CMC and mAP ...
** Results **
mAP: 55.6%
CMC curve
Rank-1  : 76.4%
Rank-5  : 89.8%
Rank-10 : 92.3%
Rank-20 : 95.0%
Epoch 25/30: Rank1: 0.7642517685890198, mAP: 0.5556826416609605
epoch: [26/30][50/404]	time 0.072 (0.085)	eta 0:02:48	Loss 0.0693 (0.1330)	
epoch: [26/30][100/404]	time 0.074 (0.079)	eta 0:02:32	Loss 0.0985 (0.1131)	
epoch: [26/30][150/404]	time 0.075 (0.077)	eta 0:02:24	Loss 0.0614 (0.1052)	
epoch: [26/30][200/404]	time 0.073 (0.077)	eta 0:02:19	Loss 0.0778 (0.1061)	
epoch: [26/30][250/404]	time 0.077 (0.076)	eta 0:02:14	Loss 0.2618 (0.1096)	
epoch: [26/30][300/404]	time 0.073 (0.076)	eta 0

# old 


In [ ]:

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation
    )


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(
        in_planes, out_planes, kernel_size=1, stride=stride, bias=False
    )



class ResNet(nn.Module):
    """Residual network.
    
    Reference:
        - He et al. Deep Residual Learning for Image Recognition. CVPR 2016.
        - Xie et al. Aggregated Residual Transformations for Deep Neural Networks. CVPR 2017.
    Public keys:
        - ``resnet18``: ResNet18.
        - ``resnet34``: ResNet34.
        - ``resnet50``: ResNet50.
        - ``resnet50_fc512``: ResNet50 + FC.
    """

    def __init__(
        self,
        num_classes,
        loss,
        block,
        layers,
        zero_init_residual=False,
        groups=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
        last_stride=2,
        fc_dims=None,
        dropout_p=None,
        **kwargs
    ):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.loss = loss
        self.feature_dim = 512 * block.expansion
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                "or a 3-element tuple, got {}".
                format(replace_stride_with_dilation)
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(
            3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(
            block,
            128,
            layers[1],
            stride=2,
            dilate=replace_stride_with_dilation[0]
        )
        self.layer3 = self._make_layer(
            block,
            256,
            layers[2],
            stride=2,
            dilate=replace_stride_with_dilation[1]
        )
        self.layer4 = self._make_layer(
            block,
            512,
            layers[3],
            stride=last_stride,
            dilate=replace_stride_with_dilation[2]
        )
        self.global_avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = self._construct_fc_layer(
            fc_dims, 512 * block.expansion, dropout_p
        )
        self.classifier = nn.Linear(self.feature_dim, num_classes)

        self._init_params()


    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups,
                self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer
                )
            )

        return nn.Sequential(*layers)

    def _construct_fc_layer(self, fc_dims, input_dim, dropout_p=None):
        """Constructs fully connected layer
        Args:
            fc_dims (list or tuple): dimensions of fc layers, if None, no fc layers are constructed
            input_dim (int): input dimension
            dropout_p (float): dropout probability, if None, dropout is unused
        """
        if fc_dims is None:
            self.feature_dim = input_dim
            return None

        assert isinstance(
            fc_dims, (list, tuple)
        ), 'fc_dims must be either list or tuple, but got {}'.format(
            type(fc_dims)
        )

        layers = []
        for dim in fc_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.BatchNorm1d(dim))
            layers.append(nn.ReLU(inplace=True))
            if dropout_p is not None:
                layers.append(nn.Dropout(p=dropout_p))
            input_dim = dim

        self.feature_dim = fc_dims[-1]

        return nn.Sequential(*layers)

    def _init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu'
                )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def featuremaps(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, x):
        f = self.featuremaps(x)
        v = self.global_avgpool(f)
        v = v.view(v.size(0), -1)

        if self.fc is not None:
            v = self.fc(v)

        if not self.training:
            return v

        y = self.classifier(v)

        if self.loss == 'softmax':
            return y
        elif self.loss == 'triplet':
            return y, v
        else:
            raise KeyError("Unsupported loss: {}".format(self.loss))



In [ ]:
baseline = [nn.Conv2d(Cin, C, kernel_size=3, stride=2, padding=1), 
    nn.ReLU(inplace=True)]
antialiased = [nn.Conv2d(Cin, C, kernel_size=3, stride=1, padding=1),
    nn.ReLU(inplace=True),
    antialiased_cnns.BlurPool(C, stride=2)]

In [ ]:
from antialiased_cnns import BlurPool

In [ ]:


class AntialiasedBasicBlock(nn.Module):
    expansion = 1

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
        filter_size=1
    ):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError(
                'BasicBlock only supports groups=1 and base_width=64'
            )
        if dilation > 1:
            raise NotImplementedError(
                "Dilation > 1 not supported in BasicBlock"
            )
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        if(stride==1):
            self.conv3 = conv1x1(width, planes * self.expansion)
        else:
            self.conv3 = nn.Sequential(BlurPool(width, filt_size=filter_size, stride=stride),
                conv1x1(width, planes * self.expansion))

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [37]:
class antalliassing_ResNet(ResNet):
    def __init__(self):
        super().__init__()
        self.base_conv1 = nn.Conv2d(
            3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(
            block,
            128,
            layers[1],
            stride=2,
            dilate=replace_stride_with_dilation[0]
        )
        self.layer3 = self._make_layer(
            block,
            256,
            layers[2],
            stride=2,
            dilate=replace_stride_with_dilation[1]
        )
        self.layer4 = self._make_layer(
            block,
            512,
            layers[3],
            stride=last_stride,
            dilate=replace_stride_with_dilation[2]
        )
        self.global_avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = self._construct_fc_layer(
            fc_dims, 512 * block.expansion, dropout_p
        )
        self.classifier = nn.Linear(self.feature_dim, num_classes)

        self._init_params()

NameError: name 'ResNet' is not defined

In [ ]:

def resnet34(num_classes, loss='softmax', pretrained=True, **kwargs):
    model = ResNet(
        num_classes=num_classes,
        loss=loss,
        block=BasicBlock,
        layers=[3, 4, 6, 3],
        last_stride=2,
        fc_dims=None,
        dropout_p=None,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, model_urls['resnet34'])
    return model

# Part II - Triplet loss and hard negative mining.

Now, we can combine both losses and train a new model.

In [12]:
class HardBatchMiningTripletLoss(torch.nn.Module):
    """Triplet loss with hard positive/negative mining of samples in a batch.
    
    Reference:
        Hermans et al. In Defense of the Triplet Loss for Person Re-Identification. arXiv:1703.07737.
    Args:
        margin (float, optional): margin for triplet. Default is 0.3.
    """

    def __init__(self, margin=0.3):
        super(HardBatchMiningTripletLoss, self).__init__()
        self.margin = margin
        self.ranking_loss = torch.nn.MarginRankingLoss(margin=margin)

    def forward(self, inputs, targets):
        """
        Args:
            inputs (torch.Tensor): feature matrix with shape (batch_size, feat_dim).
            targets (torch.LongTensor): ground truth labels with shape (batch_size).
        """
        n = inputs.size(0)

        # Compute the pairwise euclidean distance between all n feature vectors.

        #distance_matrix = euclidean_squared_distance(inputs, inputs)
        #distance_matrix = distance_matrix.clamp(min=1e-12).sqrt()
        distance_matrix = torch.cdist(inputs, inputs, p=2.0) # clear euclidian dist 
        
        # For each sample (image), find the hardest positive and hardest negative sample.
        # The targets are a vector that encode the class label for each of the n samples.
        # Pairs of samples with the SAME class can form a positive sample.
        # Pairs of samples with a DIFFERENT class can form a negative sample.
        #
        # loop over all samples, and for each one
        # find the hardest positive sample and the hardest negative sample.
        # The distances are then added to the following lists.
        # Positive pairs should be as close as possible, while 
        # negative pairs should be quite far apart. 
        mask = targets.expand(n, n).eq(targets.expand(n, n).t())
        
        distance_positive_pairs, distance_negative_pairs = [], []
        for i in range(n):
            row_dist = distance_matrix[i]
            row_mask = mask[i]
            hard_pos_dist = row_dist[row_mask].max().unsqueeze(0)
            hard_neg_dist = row_dist[row_mask==0].min().unsqueeze(0)
            distance_positive_pairs.append(hard_pos_dist)
            distance_negative_pairs.append(hard_neg_dist) 
        distance_positive_pairs = torch.cat(distance_positive_pairs)
        distance_negative_pairs = torch.cat(distance_negative_pairs)

        # The ranking loss will compute the triplet loss with the margin.
        # loss = max(0, -1*(neg_dist - pos_dist) + margin)
        y = torch.ones_like(distance_negative_pairs)
        return self.ranking_loss(distance_negative_pairs, distance_positive_pairs, y)

In [13]:
model = build_model('resnet34', datamanager.num_train_pids, loss='triplet', pretrained=True)
model = model.cuda()

trainable_params = model.parameters()
optimizer = torch.optim.Adam(trainable_params, lr=0.0003, 
                             weight_decay=5e-4, amsgrad=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)

In [14]:
class CombinedLoss(object):
    def __init__(self, margin=0.3, weight_triplet=1.0, weight_ce=1.0):
        super(CombinedLoss, self).__init__()
        self.triplet_loss = HardBatchMiningTripletLoss() 
        self.cross_entropy = torch.nn.CrossEntropyLoss()
        self.weight_triplet = weight_triplet
        self.weight_ce = weight_ce

    def __call__(self, logits, features, gt_pids):
        loss = 0.0
        loss_summary = {}
        if self.weight_triplet > 0.0:
            loss_t = self.triplet_loss(features, gt_pids) * self.weight_triplet
            loss += loss_t
            loss_summary['Triplet Loss'] = loss_t

        if self.weight_ce > 0.0:
            loss_ce = self.cross_entropy(logits, gt_pids) * self.weight_ce
            loss += loss_ce
            loss_summary['CE Loss'] = loss_ce

        loss_summary['Loss'] = loss
        return loss, loss_summary

In [15]:
MAX_EPOCH = 30
EPOCH_EVAL_FREQ = 5
PRINT_FREQ = 10

num_batches = len(train_loader)
criterion = CombinedLoss(0.3, 1.0, 1.0) 

for epoch in range(MAX_EPOCH):
    losses = utils.MetricMeter()
    batch_time = utils.AverageMeter()
    end = time.time()
    model.train()
    for batch_idx, data in enumerate(train_loader):
        # Predict output.
        imgs, pids = data['img'].cuda(), data['pid'].cuda()
        logits, features = model(imgs)
        # Compute loss.
        loss, loss_summary = criterion(logits, features, pids)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time.update(time.time() - end)
        losses.update(loss_summary)
        if (batch_idx + 1) % PRINT_FREQ == 0:
            utils.print_statistics(batch_idx, num_batches, epoch, MAX_EPOCH, batch_time, losses)
        end = time.time()
        
    if (epoch + 1) % EPOCH_EVAL_FREQ == 0 or epoch == MAX_EPOCH - 1:
        rank1, mAP = evaluate(model, test_loader)
        print('Epoch {0}/{1}: Rank1: {rank}, mAP: {map}'.format(
                    epoch + 1, MAX_EPOCH, rank=rank1, map=mAP))

epoch: [1/30][10/404]	time 0.095 (0.385)	eta 1:17:48	Triplet Loss 0.4190 (0.2325)	CE Loss 6.6653 (6.6329)	Loss 7.0843 (6.8654)	
epoch: [1/30][20/404]	time 0.094 (0.240)	eta 0:48:19	Triplet Loss 0.0000 (0.1873)	CE Loss 6.8805 (6.7433)	Loss 6.8805 (6.9306)	
epoch: [1/30][30/404]	time 0.094 (0.191)	eta 0:38:29	Triplet Loss 0.0566 (0.1655)	CE Loss 7.4746 (6.7975)	Loss 7.5312 (6.9630)	
epoch: [1/30][40/404]	time 0.094 (0.167)	eta 0:33:33	Triplet Loss 0.3055 (0.2146)	CE Loss 7.0941 (6.8215)	Loss 7.3996 (7.0361)	
epoch: [1/30][50/404]	time 0.094 (0.152)	eta 0:30:36	Triplet Loss 0.0000 (0.1781)	CE Loss 6.5143 (6.7745)	Loss 6.5143 (6.9527)	
epoch: [1/30][60/404]	time 0.095 (0.142)	eta 0:28:37	Triplet Loss 0.0000 (0.1649)	CE Loss 6.5989 (6.7645)	Loss 6.5989 (6.9294)	
epoch: [1/30][70/404]	time 0.093 (0.135)	eta 0:27:12	Triplet Loss 0.0000 (0.1543)	CE Loss 6.6829 (6.7273)	Loss 6.6829 (6.8816)	
epoch: [1/30][80/404]	time 0.096 (0.130)	eta 0:26:09	Triplet Loss 0.0020 (0.1469)	CE Loss 6.5811 (6.7052

epoch: [2/30][250/404]	time 0.097 (0.096)	eta 0:18:22	Triplet Loss 0.0000 (0.1050)	CE Loss 3.5597 (3.9212)	Loss 3.5597 (4.0262)	
epoch: [2/30][260/404]	time 0.096 (0.096)	eta 0:18:20	Triplet Loss 0.0000 (0.1033)	CE Loss 3.1959 (3.8966)	Loss 3.1959 (3.9999)	
epoch: [2/30][270/404]	time 0.095 (0.096)	eta 0:18:17	Triplet Loss 0.0163 (0.1009)	CE Loss 3.2123 (3.8693)	Loss 3.2286 (3.9702)	
epoch: [2/30][280/404]	time 0.095 (0.096)	eta 0:18:15	Triplet Loss 0.0000 (0.1008)	CE Loss 3.2472 (3.8471)	Loss 3.2472 (3.9478)	
epoch: [2/30][290/404]	time 0.090 (0.096)	eta 0:18:12	Triplet Loss 0.0000 (0.1002)	CE Loss 3.4609 (3.8211)	Loss 3.4609 (3.9213)	
epoch: [2/30][300/404]	time 0.093 (0.096)	eta 0:18:10	Triplet Loss 0.0000 (0.0975)	CE Loss 3.1401 (3.7983)	Loss 3.1401 (3.8958)	
epoch: [2/30][310/404]	time 0.096 (0.095)	eta 0:18:09	Triplet Loss 0.0000 (0.0970)	CE Loss 2.8756 (3.7741)	Loss 2.8756 (3.8711)	
epoch: [2/30][320/404]	time 0.096 (0.095)	eta 0:18:07	Triplet Loss 0.0000 (0.0942)	CE Loss 3.0727

epoch: [4/30][90/404]	time 0.097 (0.099)	eta 0:17:48	Triplet Loss 0.0000 (0.0298)	CE Loss 1.1818 (1.3659)	Loss 1.1818 (1.3956)	
epoch: [4/30][100/404]	time 0.096 (0.099)	eta 0:17:44	Triplet Loss 0.0000 (0.0303)	CE Loss 1.1995 (1.3560)	Loss 1.1995 (1.3862)	
epoch: [4/30][110/404]	time 0.097 (0.098)	eta 0:17:41	Triplet Loss 0.0000 (0.0284)	CE Loss 1.0889 (1.3531)	Loss 1.0889 (1.3815)	
epoch: [4/30][120/404]	time 0.097 (0.098)	eta 0:17:39	Triplet Loss 0.0000 (0.0265)	CE Loss 1.2470 (1.3490)	Loss 1.2470 (1.3755)	
epoch: [4/30][130/404]	time 0.097 (0.098)	eta 0:17:36	Triplet Loss 0.0000 (0.0254)	CE Loss 1.3735 (1.3592)	Loss 1.3735 (1.3846)	
epoch: [4/30][140/404]	time 0.097 (0.098)	eta 0:17:34	Triplet Loss 0.0000 (0.0236)	CE Loss 1.2433 (1.3613)	Loss 1.2433 (1.3849)	
epoch: [4/30][150/404]	time 0.096 (0.098)	eta 0:17:32	Triplet Loss 0.0000 (0.0225)	CE Loss 1.0470 (1.3576)	Loss 1.0470 (1.3802)	
epoch: [4/30][160/404]	time 0.097 (0.098)	eta 0:17:30	Triplet Loss 0.0000 (0.0220)	CE Loss 1.1870 

epoch: [5/30][330/404]	time 0.096 (0.096)	eta 0:16:19	Triplet Loss 0.0000 (0.0107)	CE Loss 1.1861 (0.7909)	Loss 1.1861 (0.8017)	
epoch: [5/30][340/404]	time 0.094 (0.096)	eta 0:16:17	Triplet Loss 0.0000 (0.0113)	CE Loss 0.7691 (0.7916)	Loss 0.7691 (0.8029)	
epoch: [5/30][350/404]	time 0.097 (0.096)	eta 0:16:16	Triplet Loss 0.0000 (0.0113)	CE Loss 0.8311 (0.7907)	Loss 0.8311 (0.8020)	
epoch: [5/30][360/404]	time 0.094 (0.096)	eta 0:16:15	Triplet Loss 0.0000 (0.0128)	CE Loss 0.5735 (0.7895)	Loss 0.5735 (0.8023)	
epoch: [5/30][370/404]	time 0.097 (0.096)	eta 0:16:14	Triplet Loss 0.0000 (0.0132)	CE Loss 0.5007 (0.7868)	Loss 0.5007 (0.8001)	
epoch: [5/30][380/404]	time 0.096 (0.096)	eta 0:16:13	Triplet Loss 0.0000 (0.0131)	CE Loss 0.6731 (0.7875)	Loss 0.6731 (0.8006)	
epoch: [5/30][390/404]	time 0.096 (0.096)	eta 0:16:12	Triplet Loss 0.0595 (0.0131)	CE Loss 0.9138 (0.7898)	Loss 0.9733 (0.8029)	
epoch: [5/30][400/404]	time 0.096 (0.096)	eta 0:16:11	Triplet Loss 0.0000 (0.0128)	CE Loss 0.7368

epoch: [7/30][150/404]	time 0.088 (0.098)	eta 0:15:31	Triplet Loss 0.0000 (0.0205)	CE Loss 0.5598 (0.4939)	Loss 0.5598 (0.5144)	
epoch: [7/30][160/404]	time 0.089 (0.097)	eta 0:15:27	Triplet Loss 0.0000 (0.0192)	CE Loss 0.4854 (0.4904)	Loss 0.4854 (0.5096)	
epoch: [7/30][170/404]	time 0.088 (0.097)	eta 0:15:23	Triplet Loss 0.0000 (0.0197)	CE Loss 0.4561 (0.4910)	Loss 0.4561 (0.5107)	
epoch: [7/30][180/404]	time 0.087 (0.097)	eta 0:15:19	Triplet Loss 0.0000 (0.0186)	CE Loss 0.3798 (0.4878)	Loss 0.3798 (0.5064)	
epoch: [7/30][190/404]	time 0.090 (0.096)	eta 0:15:15	Triplet Loss 0.0000 (0.0176)	CE Loss 0.5047 (0.4864)	Loss 0.5047 (0.5040)	
epoch: [7/30][200/404]	time 0.097 (0.096)	eta 0:15:13	Triplet Loss 0.0000 (0.0167)	CE Loss 0.5017 (0.4845)	Loss 0.5017 (0.5012)	
epoch: [7/30][210/404]	time 0.096 (0.096)	eta 0:15:12	Triplet Loss 0.0000 (0.0159)	CE Loss 0.3081 (0.4802)	Loss 0.3081 (0.4961)	
epoch: [7/30][220/404]	time 0.096 (0.096)	eta 0:15:11	Triplet Loss 0.0000 (0.0152)	CE Loss 0.5109

...

epoch: [24/30][300/404]	time 0.096 (0.096)	eta 0:04:02	Triplet Loss 0.0000 (0.0015)	CE Loss 0.1227 (0.1094)	Loss 0.1227 (0.1109)	
epoch: [24/30][310/404]	time 0.095 (0.096)	eta 0:04:01	Triplet Loss 0.0000 (0.0015)	CE Loss 0.1152 (0.1099)	Loss 0.1152 (0.1114)	
epoch: [24/30][320/404]	time 0.097 (0.096)	eta 0:04:00	Triplet Loss 0.0000 (0.0019)	CE Loss 0.1401 (0.1099)	Loss 0.1401 (0.1118)	
epoch: [24/30][330/404]	time 0.097 (0.096)	eta 0:04:00	Triplet Loss 0.0000 (0.0018)	CE Loss 0.2892 (0.1112)	Loss 0.2892 (0.1130)	
epoch: [24/30][340/404]	time 0.096 (0.096)	eta 0:03:59	Triplet Loss 0.0000 (0.0017)	CE Loss 0.1266 (0.1114)	Loss 0.1266 (0.1132)	
epoch: [24/30][350/404]	time 0.095 (0.096)	eta 0:03:58	Triplet Loss 0.0000 (0.0017)	CE Loss 0.1518 (0.1135)	Loss 0.1518 (0.1152)	
epoch: [24/30][360/404]	time 0.096 (0.096)	eta 0:03:57	Triplet Loss 0.0000 (0.0017)	CE Loss 0.1380 (0.1134)	Loss 0.1380 (0.1150)	
epoch: [24/30][370/404]	time 0.096 (0.096)	eta 0:03:56	Triplet Loss 0.0000 (0.0021)	CE Los

epoch: [26/30][110/404]	time 0.095 (0.095)	eta 0:03:02	Triplet Loss 0.0000 (0.0002)	CE Loss 0.0804 (0.1077)	Loss 0.0804 (0.1080)	
epoch: [26/30][120/404]	time 0.094 (0.095)	eta 0:03:01	Triplet Loss 0.0000 (0.0002)	CE Loss 0.0878 (0.1072)	Loss 0.0878 (0.1074)	
epoch: [26/30][130/404]	time 0.088 (0.095)	eta 0:02:59	Triplet Loss 0.0000 (0.0002)	CE Loss 0.0669 (0.1070)	Loss 0.0669 (0.1072)	
epoch: [26/30][140/404]	time 0.094 (0.095)	eta 0:02:58	Triplet Loss 0.0000 (0.0002)	CE Loss 0.1684 (0.1070)	Loss 0.1684 (0.1072)	
epoch: [26/30][150/404]	time 0.095 (0.095)	eta 0:02:57	Triplet Loss 0.0000 (0.0002)	CE Loss 0.1687 (0.1073)	Loss 0.1687 (0.1074)	
epoch: [26/30][160/404]	time 0.087 (0.094)	eta 0:02:55	Triplet Loss 0.0000 (0.0002)	CE Loss 0.0944 (0.1069)	Loss 0.0944 (0.1071)	
epoch: [26/30][170/404]	time 0.094 (0.094)	eta 0:02:54	Triplet Loss 0.0000 (0.0001)	CE Loss 0.0623 (0.1057)	Loss 0.0623 (0.1059)	
epoch: [26/30][180/404]	time 0.088 (0.094)	eta 0:02:53	Triplet Loss 0.0000 (0.0004)	CE Los

epoch: [27/30][350/404]	time 0.096 (0.095)	eta 0:02:00	Triplet Loss 0.0381 (0.0008)	CE Loss 0.1424 (0.0995)	Loss 0.1805 (0.1003)	
epoch: [27/30][360/404]	time 0.096 (0.095)	eta 0:01:59	Triplet Loss 0.0000 (0.0010)	CE Loss 0.0711 (0.1011)	Loss 0.0711 (0.1021)	
epoch: [27/30][370/404]	time 0.097 (0.095)	eta 0:01:58	Triplet Loss 0.0000 (0.0009)	CE Loss 0.2152 (0.1039)	Loss 0.2152 (0.1048)	
epoch: [27/30][380/404]	time 0.096 (0.095)	eta 0:01:57	Triplet Loss 0.0000 (0.0010)	CE Loss 0.1429 (0.1058)	Loss 0.1429 (0.1068)	
epoch: [27/30][390/404]	time 0.097 (0.095)	eta 0:01:56	Triplet Loss 0.0000 (0.0010)	CE Loss 0.3094 (0.1076)	Loss 0.3094 (0.1086)	
epoch: [27/30][400/404]	time 0.096 (0.095)	eta 0:01:56	Triplet Loss 0.0000 (0.0012)	CE Loss 0.1100 (0.1086)	Loss 0.1100 (0.1098)	
epoch: [28/30][10/404]	time 0.096 (0.120)	eta 0:02:23	Triplet Loss 0.0000 (0.0031)	CE Loss 0.1187 (0.1356)	Loss 0.1187 (0.1387)	
epoch: [28/30][20/404]	time 0.097 (0.108)	eta 0:02:08	Triplet Loss 0.0000 (0.0015)	CE Loss 

epoch: [29/30][190/404]	time 0.097 (0.096)	eta 0:00:59	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0454 (0.0543)	Loss 0.0454 (0.0543)	
epoch: [29/30][200/404]	time 0.099 (0.096)	eta 0:00:58	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0548 (0.0542)	Loss 0.0548 (0.0542)	
epoch: [29/30][210/404]	time 0.097 (0.096)	eta 0:00:57	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0423 (0.0538)	Loss 0.0423 (0.0538)	
epoch: [29/30][220/404]	time 0.098 (0.096)	eta 0:00:56	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0195 (0.0536)	Loss 0.0195 (0.0536)	
epoch: [29/30][230/404]	time 0.098 (0.096)	eta 0:00:55	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0507 (0.0535)	Loss 0.0507 (0.0535)	
epoch: [29/30][240/404]	time 0.097 (0.096)	eta 0:00:54	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0257 (0.0533)	Loss 0.0257 (0.0533)	
epoch: [29/30][250/404]	time 0.097 (0.096)	eta 0:00:53	Triplet Loss 0.0000 (0.0000)	CE Loss 0.0282 (0.0544)	Loss 0.0282 (0.0544)	
epoch: [29/30][260/404]	time 0.097 (0.096)	eta 0:00:52	Triplet Loss 0.0000 (0.0000)	CE Los

** Results **
mAP: 50.1%
CMC curve
Rank-1  : 72.0%
Rank-5  : 86.9%
Rank-10 : 91.2%
Rank-20 : 94.1%
Epoch 30/30: Rank1: 0.7197149395942688, mAP: 0.5013516446733468


## Save model

In [16]:
model_path = reid_root_dir+'/models/resnet34_reid_market.model'
model_path

'../models/resnet34_reid_market.model'

In [17]:
torch.save(model.state_dict(), model_path)


## test load

In [18]:
model = build_model('resnet34', datamanager.num_train_pids, loss='triplet', pretrained=True)
model = model.cuda()


In [19]:
reid_market_state_dict = torch.load(model_path,
                                   map_location=lambda storage, loc: storage)

In [20]:
model.load_state_dict(reid_market_state_dict)

<All keys matched successfully>

## eval model

In [21]:
rank1, mAP = evaluate(model, test_loader)

Extracting features from query set...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Computing CMC and mAP ...
** Results **
mAP: 50.1%
CMC curve
Rank-1  : 72.0%
Rank-5  : 86.9%
Rank-10 : 91.2%
Rank-20 : 94.1%
